In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,63,2024-03-07,1,113.338710,96.620968,0.562710,12.754839,23.116129,0.222952,116.666129,...,11.114286,23.696825,0.176968,121.587302,0,3,3,3,18,2024
1,63,2024-03-07,1,112.000000,96.667742,0.535452,10.738710,24.791935,0.175629,115.237097,...,12.952459,24.621311,0.195689,112.142623,0,3,3,2,3,2024
2,62,2024-03-07,2,110.754098,96.018033,0.538377,11.263934,22.183607,0.222230,115.178689,...,10.616129,22.153226,0.198145,119.032258,0,3,3,2,16,2024
3,63,2024-03-07,-1,114.129032,98.690323,0.541726,11.312903,25.011290,0.183952,114.940323,...,12.653226,24.783871,0.239984,118.782258,0,3,3,2,28,2024
4,62,2024-03-07,-1,121.000000,97.224590,0.577590,10.542623,24.381967,0.195525,123.611475,...,10.822581,25.912903,0.175452,119.083871,0,3,3,2,2,2024
5,63,2024-03-07,2,111.790323,95.388710,0.531565,10.498387,24.459677,0.187806,115.327419,...,12.095082,27.340984,0.183098,118.524590,0,3,3,1,4,2024
6,63,2024-03-07,-1,112.161290,100.887097,0.531613,12.633871,22.740323,0.171742,110.930645,...,11.263934,23.891803,0.171246,118.209836,0,3,3,2,27,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0
1,2024-03-07,BRK,DET,1.769231,2.100000,1
2,2024-03-07,MIA,DAL,2.600000,1.520833,0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0
4,2024-03-07,BOS,DEN,1.952381,1.869565,0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0
1,2024-03-07,BRK,DET,1.769231,2.100000,0
2,2024-03-07,MIA,DAL,2.600000,1.520833,0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0
4,2024-03-07,BOS,DEN,1.952381,1.869565,0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0
1,2024-03-07,BRK,DET,1.769231,2.100000,0
2,2024-03-07,MIA,DAL,2.600000,1.520833,0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0
4,2024-03-07,BOS,DEN,1.952381,1.869565,0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0
1,2024-03-07,BRK,DET,1.769231,2.100000,1
2,2024-03-07,MIA,DAL,2.600000,1.520833,0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0
4,2024-03-07,BOS,DEN,1.952381,1.869565,0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0.0
1,2024-03-07,BRK,DET,1.769231,2.100000,1.0
2,2024-03-07,MIA,DAL,2.600000,1.520833,0.0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0.0
4,2024-03-07,BOS,DEN,1.952381,1.869565,1.0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0.0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-07,MIN,IND,2.000000,1.833333,0.0
1,2024-03-07,BRK,DET,1.769231,2.100000,1.0
2,2024-03-07,MIA,DAL,2.600000,1.520833,0.0
3,2024-03-07,TOR,PHO,4.800000,1.200000,0.0
4,2024-03-07,BOS,DEN,1.952381,1.869565,1.0
5,2024-03-07,CHI,GSW,3.700000,1.294118,0.0
6,2024-03-07,SAS,SAC,5.250000,1.173913,0.0
